In [1]:
import pandas as pd
import geopandas as gpd
from functools import partial
import numpy as np

from ipyleaflet import Map, Marker, Popup
from IPython.display import display
from ipywidgets import HTML
import ipywidgets as widgets
import folium


In [5]:
# Read CSV file
csv_file = 'data/sensor_graph/denhaag_locations_type.csv'
df = pd.read_csv(csv_file)
print(df.head())
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.latitude, df.longitude), crs="EPSG:4326")

   index  sensor_id  latitude  longitude  type  direction  code
0      0     120022  52.09082    4.30510     1         -1   120
1      1     120023  52.09082    4.30510     1          1   120
2      2     120051  52.09073    4.30464     0         -1   120
3      3     120081  52.09101    4.30479     1          0   120
4      4     120111  52.09097    4.30508     0          1   120


In [6]:
# Initialize an empty list to store selected nodes *SET DEFAULT
selected_node_index = 30

# Function to create a popup with node information
def create_popup(row):
    popup = Popup(
        location=(row['latitude'], row['longitude']),
        child=HTML(
            value=f"Index: {row['index']}<br>"
                  f"Sensor ID: {row['sensor_id']}<br>"
                  f"Latitude: {row['latitude']}<br>"
                  f"Longitude: {row['longitude']}"
        ),
        close_button=True,
        auto_close=False,
        close_on_escape_key=False
    )
    return popup

# Function to handle interactive node selection
def on_marker_click(marker):
    global selected_node_index 
    selected_node = marker.row_data
    selected_node.pop('distance', None)  # Remove the 'distance' key
    selected_node.pop('geometry', None)  # Remove the 'geometry' key
    #selected_nodes.append(selected_node)
    index = selected_node['index']
    selected_node_index = int(index) 
    print(f"Selected node: {selected_node}")

# Wrapper function to pass the marker object to the on_marker_click function
def create_click_callback(marker):
    def click_callback(*args, **kwargs):
        on_marker_click(marker)
    return click_callback

# Create an ipyleaflet map
map_center = [gdf.latitude.mean(), gdf.longitude.mean()]
m = Map(center=map_center, zoom=11)

# Add nodes to the map
for idx, row in gdf.iterrows():
    marker = Marker(location=[row['latitude'], row['longitude']])
    marker.popup = create_popup(row)
    marker.row_data = row.to_dict()

    # Use the wrapper function to create a callback for the current marker
    click_callback = create_click_callback(marker)
    marker.on_click(click_callback)
    m.add_layer(marker)

# Display the map
display(m)

Map(center=[52.09175666666667, 4.291986991869918], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [7]:
n_nearest_nodes = 123

# Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    return R * c

# Get the selected node's latitude and longitude
sn_lat = df.latitude[selected_node_index]
sn_lon = df.longitude[selected_node_index]

# Calculate the distance from the selected node to all other nodes in the DataFrame
df['distance'] = df.apply(lambda row: haversine(sn_lat, sn_lon, row['latitude'], row['longitude']), axis=1)

# Get the n_nearest_nodes with the smallest 'distance' values
selected_nodes_df = df.nsmallest(n_nearest_nodes, 'distance')

# Check if 'distance' column exists in subset_df and drop it if it does
if 'distance' in selected_nodes_df.columns:
    selected_nodes_df.drop(columns=['distance'], inplace=True)

# Check if 'geometry' column exists in subset_df and drop it if it does
if 'geometry' in selected_nodes_df.columns:
    selected_nodes_df.drop(columns=['geometry'], inplace=True)

# Check if 'distance' column exists in subset_df and drop it if it does
if 'distance' in df.columns:
    df.drop(columns=['distance'], inplace=True)

# Check if 'geometry' column exists in subset_df and drop it if it does
if 'geometry' in df.columns:
    df.drop(columns=['geometry'], inplace=True)

selected_nodes_df = selected_nodes_df.sort_values('index')

# Print the selection of nodes
print(selected_nodes_df)


     index  sensor_id  latitude  longitude  type  direction  code
0        0     120022  52.09082    4.30510     1         -1   120
1        1     120023  52.09082    4.30510     1          1   120
2        2     120051  52.09073    4.30464     0         -1   120
3        3     120081  52.09101    4.30479     1          0   120
4        4     120111  52.09097    4.30508     0          1   120
..     ...        ...       ...        ...   ...        ...   ...
118    118     712753  52.09608    4.28736     0          1   712
119    119     712754  52.09609    4.28733    -1          1   712
120    120     710801  52.09529    4.28563    -1         -1   710
121    121     710811  52.09528    4.28569     0         -1   710
122    122     710812  52.09525    4.28571     0         -1   710

[123 rows x 7 columns]


In [8]:
import folium

# Make new Geopandas df
sgdf = gpd.GeoDataFrame(selected_nodes_df, geometry=gpd.points_from_xy(selected_nodes_df.latitude, selected_nodes_df.longitude), crs="EPSG:4326")

# Create a folium map
s_map_center = [sgdf.latitude.mean(), sgdf.longitude.mean()]
fm = folium.Map(location=s_map_center, zoom_start=14)

# Add nodes to the map
for idx, row in sgdf.iterrows():
    marker = folium.Marker(location=[row['latitude'], row['longitude']])
    
    # Create a popup with node properties
    props = f"Node ID: {row['index']}<br>Latitude: {row['latitude']}<br>Longitude: {row['longitude']}"
    popup = folium.Popup(props, max_width=300)
    marker.add_child(popup)

    # Add a permanent text label with the node index
    folium.map.Marker(
        [row['latitude'], row['longitude']],
        icon=folium.DivIcon(
            icon_size=(150, 36),
            icon_anchor=(10, -5),  # Adjust the position of the text relative to the marker
            html=f'<div style="font-size: 12pt; color: black;">{row["index"]}</div>',
        ),
    ).add_to(fm)

    # Add the marker to the map
    fm.add_child(marker)

# Display the map
display(fm)


In [9]:
indices_list = selected_nodes_df.index.tolist()
print(f"List of indices: {indices_list}")

# Convert the list of indices to a comma-separated string
ids_str = ','.join(map(str, indices_list))

# Save the string to a text file
with open('data/sensor_graph/graph_indices.txt', 'w') as file:
    file.write(ids_str)


List of indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122]


In [10]:
sensor_id_list = selected_nodes_df.sensor_id.tolist()
print(f"List of sensor_ids: {sensor_id_list}")

# Convert the list of indices to a comma-separated string
sensor_ids_str = ','.join(map(str, sensor_id_list))

# Save the string to a text file
with open('data/sensor_graph/graph_sensor_ids.txt', 'w') as file:
    file.write(sensor_ids_str)


List of sensor_ids: [120022, 120023, 120051, 120081, 120111, 120112, 120113, 134021, 134031, 134041, 134051, 134081, 134111, 134112, 140031, 140041, 140051, 140111, 140121, 159051, 159052, 159082, 159111, 159112, 159651, 159652, 159711, 159712, 159713, 405021, 405022, 405031, 405032, 405041, 405051, 405071, 405081, 405082, 405091, 405101, 405111, 405641, 405651, 405661, 405681, 405711, 405712, 405721, 406011, 406021, 406022, 406081, 406082, 406091, 406101, 406121, 701021, 701051, 701052, 701111, 701112, 701113, 701121, 702011, 702031, 702041, 702051, 702052, 702111, 702112, 702121, 703021, 703022, 703031, 703051, 703052, 703081, 703082, 703091, 703111, 703112, 709011, 709021, 709081, 709082, 709083, 709101, 709111, 709112, 709121, 709651, 709652, 709661, 709711, 709712, 711021, 711022, 710051, 710052, 710053, 711081, 712111, 712112, 712113, 712181, 712182, 711651, 711652, 711653, 711661, 710681, 710682, 711711, 711712, 711713, 711721, 712751, 712752, 712753, 712754, 710801, 710811, 710

In [11]:
# Convert the selected nodes to a DataFrame and save to a CSV file
selected_nodes_df.to_csv('data/sensor_graph/graph_sensor_locations.csv', index=False)